In [ ]:
# This file is part of the prkng_mtl project.
#
# Copyright (C) 2025 coklacour
#
# The prkng_mtl project is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# The prkng_mtl project is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

Solution existante : https://donnees.montreal.ca/showcase/carte-curblr-de-montreal

## Packages

In [ ]:
import os
import pandas as pd
import numpy as np

import folium
from folium import plugins

## Macro variables

In [ ]:
INPUT_DIR = os.path.dirname(os.path.abspath(''))+'/datas/raw/'
OUTPUT_DIR= os.path.dirname(os.path.abspath(''))+'/reports/'

## Import

Park

In [ ]:
park = pd.read_csv(INPUT_DIR+"/park.csv", sep=',')

Geobase

In [ ]:
geobase = pd.read_csv(INPUT_DIR+"/geobase.csv", sep=',')

Geobase double

In [ ]:
double = pd.read_csv(INPUT_DIR+"/double.csv", sep=',')

Geobase noeuds

In [ ]:
nodes = pd.read_csv(INPUT_DIR+"/nodes.csv", sep=',')

## Exploration

### Panneaux

#### Manipulation

focus sur les panneaux du plateau mont royal

In [ ]:
park['NOM_ARROND'].unique()

In [ ]:
plateau = park[park['NOM_ARROND'] == 'Le Plateau-Mont-Royal']
plateau.head(2)

Y'a t'il des categories de panneaux a ne pas considerer dans le cadre de ce projet?

In [ ]:
plateau['DESCRIPTION_CAT'].unique()

In [ ]:
plateau.columns

In [ ]:
print("Nombre de lignes concernant le Plateau :", plateau.shape[0])
print("Nombre de poteaux sur le Plateau :", len(plateau['POTEAU_ID_POT'].unique()))
print("Nombre de panneaux sur le Plateau :", len(plateau['PANNEAU_ID_PAN'].unique()))

In [ ]:
#park.groupby('POTEAU_ID_POT').size().sort_values(ascending=False)
#plateau.groupby('POTEAU_ID_POT').size().loc[lambda x: x == 2]

In [ ]:
plateau[plateau['POTEAU_ID_POT'] == 305837]

#### Vizualisation (pour le fun)

In [ ]:
res = plateau[['POTEAU_ID_POT','Longitude','Latitude','X','Y','NOM_ARROND', 'DESCRIPTION_CAT']].drop_duplicates()

In [ ]:
# dictionnaire qui distingue les categories de stationement
color_park_cat = {
    'STATIONNEMENT': 'green',
    'STAT-PANNONC.': 'orange',
    'STAT-$': 'red'
}

# Centrer sur le Plateau
m = folium.Map(location=[45.5271, -73.5804],  # Coordonnées du Plateau
               tiles='cartodbpositron',       # Type de carte
               min_zoom=10, max_zoom=20,      # Plage de zoom
               zoom_start=14)                 # Zoom initial pour se concentrer sur le Plateau

# Créer un MarkerCluster
marker_cluster = plugins.MarkerCluster().add_to(m)

# Boucle à travers les données du DataFrame 'park' et ajouter des marqueurs
for idx, row in res.iterrows():
    
    # Contenu HTML pour le popup
    popup_content = f"""
    <div style="width: 200px; height: 100px;">
        <b>poteau_id_pot:</b> {row['POTEAU_ID_POT']}<br>
        <b>Catégorie panneau:</b> {row['DESCRIPTION_CAT']}<br>
    </div>
    """

    popup = folium.Popup(popup_content, max_width=300)  # Largeur de la fenêtre popup

    # Ajuster la couleur selon le type de stationement
    icon_color = color_park_cat.get(row['DESCRIPTION_CAT'], 'gray') 
    
    # Ajout du marqueur avec le popup personnalisé
    folium.Marker(location=[row['Latitude'], row['Longitude']],             
                  popup=popup,  
                  icon=folium.Icon(color=icon_color)).add_to(marker_cluster)

# Afficher la carte
#m

In [ ]:
html_map = m._repr_html_()
with open(OUTPUT_DIR+"/panneaux.html", "w") as file:
    file.write(html_map)

### Geobase

Focus sur la rue Mentana

In [ ]:
geo_mentana = geobase[geobase['NOM_VOIE'].str.contains('Mentana')]
geo_mentana.head(2)


In [ ]:
geo_mentana[geo_mentana['ID_TRC'] == 1220158]

In [ ]:
dbl_mentana = double[double['NOM_VOIE'].str.contains('Mentana')]
dbl_mentana[dbl_mentana['ID_TRC'] == 1220158]

In [ ]:
nodes.head(2)
#[nodes['ID_VOIE'] == 301416]
#nodes_mentana = nodes[nodes['NOM_VOIE'].str.contains('Mentana')]
#nodes_mentana[nodes_mentana['ID_TRC'] == 1220158]